In [82]:
import requests
import json
import time
import random

In [83]:
# https://github.com/philipperemy/name-dataset
# !pip3 install names-dataset

In [84]:
from names_dataset import NameDataset, NameWrapper

In [85]:
api_key = "16831ee8bemshcddf2990350ca81p1d6a5fjsnbc1bd82e3091"

In [86]:
nd = NameDataset() 

In [87]:
# config
name_value = 3 # value of popular names used in search (for man and woman separetly)
sleep_need = False
sleep_time = 0.5 
logging = True
regions_description_file = 'regions.json'

In [88]:
# user data download 

In [89]:
# load country data 
try:
    with open(regions_description_file, 'r') as voivodeships:
        voivodeships_data = json.load(voivodeships)
except Exception as e:
    print(e)

In [90]:
voivodeships_data

{'PL': [{'name': 'Mazowsze', 'cities': ['warsaw', 'lodz']},
  {'name': 'Wielkopolska', 'cities': ['poznan', 'szczecin']},
  {'name': 'Małopolska', 'cities': ['krakow', 'rzeszow']},
  {'name': 'Śląsk', 'cities': ['katowice', 'gliwice']},
  {'name': 'Pomorze', 'cities': ['gdansk', 'gdynia']}],
 'GB': [{'name': 'England', 'cities': ['london', 'birmingham']},
  {'name': 'Scotland', 'cities': ['glasgow', 'edinburgh']},
  {'name': 'Wales', 'cities': ['cardiff', 'swansea']},
  {'name': 'Northern Ireland', 'cities': ['belfast', 'londonderry']}]}

In [91]:
users = {}

In [92]:
for country_key in list(voivodeships_data.keys()):
    users.update({country_key : {} })
    for voivodeship in voivodeships_data[country_key]:
        users[country_key].update({voivodeship["name"] : {}})
        for city in voivodeship["cities"]:
            users[country_key][voivodeship["name"]].update({city :[]})   

In [93]:
users

{'PL': {'Mazowsze': {'warsaw': [], 'lodz': []},
  'Wielkopolska': {'poznan': [], 'szczecin': []},
  'Małopolska': {'krakow': [], 'rzeszow': []},
  'Śląsk': {'katowice': [], 'gliwice': []},
  'Pomorze': {'gdansk': [], 'gdynia': []}},
 'GB': {'England': {'london': [], 'birmingham': []},
  'Scotland': {'glasgow': [], 'edinburgh': []},
  'Wales': {'cardiff': [], 'swansea': []},
  'Northern Ireland': {'belfast': [], 'londonderry': []}}}

In [94]:
def get_popular_names(country, name_value=name_value, nd=nd):
    top_names = nd.get_top_names(n=name_value, gender='Male', country_alpha2=country)
    top_names[country]["F"] = nd.get_top_names(n=name_value, gender='Female', country_alpha2=country)[country]["F"]
    return top_names

In [95]:
def get_users(city, name, offset=None, api_key=api_key):
    url = "https://soundcloud-scraper.p.rapidapi.com/v1/search/users"

    if not offset:
        querystring = {"term":name,"place":city}
    else:
        querystring = {"term":name,"offset":offset,"place":city}
    
    headers = {
    	"X-RapidAPI-Key": api_key,
    	"X-RapidAPI-Host": "soundcloud-scraper.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    return response.json()

In [96]:
get_users('warszawa', 'piotr')

{'status': True,
 'totalCount': 153,
 'users': {'nextOffset': 50,
  'items': [{'type': 'user',
    'id': 595446159,
    'permalink': 'https://soundcloud.com/piotrcartman',
    'createdAt': '2019-02-25T21:48:24Z',
    'lastModified': '2019-05-16T17:01:06Z',
    'name': 'Piotr Cartman',
    'description': 'HARDKOR/DEPRA/MROK/FILTH',
    'avatarUrl': 'https://i1.sndcdn.com/avatars-000628022391-5jmhbp-original.jpg',
    'visuals': [{'url': 'https://i1.sndcdn.com/visuals-000595446159-xZ6rlS-original.jpg',
      'entryTime': 0}],
    'followerCount': 1835,
    'followingCount': 0,
    'trackCount': 5,
    'playlistCount': 0,
    'likeCount': 1,
    'playlistLikeCount': 0,
    'commentCount': 0,
    'repostCount': None,
    'groupCount': 0,
    'city': 'Warszawa',
    'countryCode': 'PL',
    'verified': False,
    'pro': False,
    'proUnlimited': False,
    'stationPermalink': 'https://soundcloud.com/discover/sets/artist-stations:595446159'},
   {'type': 'user',
    'id': 290450290,
    'pe

In [97]:
def get_users_for_country(users, country, top_names, offset=None, api_key=api_key): 
    names = top_names[country]['M'] + top_names[country]['F']
    for voivodeship in users[country].keys():
        for city in users[country][voivodeship].keys():
            for name in names:
                print(f"looking for users with name {name} in {city}")
                users_result = get_users(city, name)
                if 'totalCount' in users_result:
                    cnt = users_result['totalCount']
                    print(f"got {cnt if cnt <= 50 else 50} of maximum {cnt} possible" )
                    if name in top_names[country]['M']:
                        users[country][voivodeship][city] += clean_users(users_result["users"]["items"], gender='M')
                    else:
                        users[country][voivodeship][city] += clean_users(users_result["users"]["items"], gender='F')
                # save_to_file(f"users_{name}_{city}_{country}", users)    
                    if sleep_need:
                        time.sleep(sleep_time)
        
    return users

In [98]:
def clean_users(users, gender=None, keys=['createdAt', 'lastModified', 'description', 'avatarUrl', 'visuals', 'followerCount', 'followingCount', 'trackCount', 'playlistCount', 'playlistLikeCount', 'commentCount', 'repostCount', 'groupCount',  'verified', 'pro', 'proUnlimited', 'stationPermalink']):
    users_result = []
    for user in users:
        for key in keys:
            if key in user:
                del user[key]
        user['gender'] = gender
        users_result.append(user)
    return users_result

In [99]:
def clean_user_likes(likes, rm_keys=['permalink', 'createdAt', 'lastModified', 'caption', 'description', 'goPlus', 'durationMs', 'durationText', 'artworkUrl', 'releaseDate', 'labelName', 'license', 'purchaseTitle', 'purchaseUrl', 'commentable', 'commentCount', 'playCount', 'repostCount', 'waveformUrl', 'stationPermalink', 'visuals', 'publisher', 'user']):
    user_likes = []    
    for like in likes:
        try:
            if like["item"]["type"] == "track":
                new_like = {
                    "type":like["type"],
                    "createdAt":like["createdAt"]
                }
                for key in like["item"].keys():
                    if key not in rm_keys:
                        new_like[key] = like["item"][key]
                if len(new_like) > 2:
                    user_likes.append(new_like)
        except Exception as e:
            print(e)
    return user_likes

In [100]:
def get_user_likes(userlinkname, offset=None, api_key=api_key):
    url = "https://soundcloud-scraper.p.rapidapi.com/v1/user/likes"    
    querystring = {"user":f"https://soundcloud.com/{userlinkname}"}    
    headers = {
    	"X-RapidAPI-Key": api_key,
    	"X-RapidAPI-Host": "soundcloud-scraper.p.rapidapi.com"
    }    
    response = requests.get(url, headers=headers, params=querystring)

    return response.json()


In [101]:
def get_users_likes(users, likes, user_city_limit = None):
    i = 1
    for country in users.keys():
        for voivideship in users[country].keys():
            for city in users[country][voivideship].keys():
                user_num = 0
                search_users = users[country][voivideship][city]
                if user_city_limit is not None:
                    random.shuffle(users[country][voivideship][city])     
                print(country, " ", city)
                for user in search_users:
                    userlinkname = user["permalink"].split('/')[-1]
                    print(f"user {userlinkname}")
                    user_likes = get_user_likes(userlinkname)
                    print(f"likes of user no: {i} , {len(user_likes)}")
                    i+=1
                    if sleep_need:
                        time.sleep(sleep_time)
                    if user_likes != None:  
                        if 'likes' in user_likes:
                            likes[userlinkname] = clean_user_likes(user_likes["likes"]["items"]) 
                            if i % 10 == 0 and logging:    
                                save_to_file(f"likes_{i}_{userlinkname}_{city}_{country}", likes)
                    if user_city_limit is not None:
                        if user_num >= user_city_limit: 
                            print("user city limit reached")
                            break
                        user_num += 1 
                    
    return likes        

In [102]:
def save_to_file(filename, file_to_save):
    with open(f"{filename}.json", "w") as outfile: 
        json.dump(file_to_save, outfile, indent = 4)

In [103]:
users.keys()

dict_keys(['PL', 'GB'])

In [104]:
users

{'PL': {'Mazowsze': {'warsaw': [], 'lodz': []},
  'Wielkopolska': {'poznan': [], 'szczecin': []},
  'Małopolska': {'krakow': [], 'rzeszow': []},
  'Śląsk': {'katowice': [], 'gliwice': []},
  'Pomorze': {'gdansk': [], 'gdynia': []}},
 'GB': {'England': {'london': [], 'birmingham': []},
  'Scotland': {'glasgow': [], 'edinburgh': []},
  'Wales': {'cardiff': [], 'swansea': []},
  'Northern Ireland': {'belfast': [], 'londonderry': []}}}

In [105]:
users = get_users_for_country(users, 'PL', get_popular_names("PL"))

looking for users with name Piotr in warsaw
got 50 of maximum 412 possible
looking for users with name Marcin in warsaw
got 50 of maximum 392 possible
looking for users with name Krzysztof in warsaw
got 50 of maximum 162 possible
looking for users with name Anna in warsaw
got 50 of maximum 185 possible
looking for users with name Agnieszka in warsaw
got 50 of maximum 87 possible
looking for users with name Katarzyna in warsaw
got 50 of maximum 96 possible
looking for users with name Piotr in lodz
got 50 of maximum 120 possible
looking for users with name Marcin in lodz
got 50 of maximum 96 possible
looking for users with name Krzysztof in lodz
got 50 of maximum 52 possible
looking for users with name Anna in lodz
got 49 of maximum 49 possible
looking for users with name Agnieszka in lodz
got 25 of maximum 25 possible
looking for users with name Katarzyna in lodz
got 22 of maximum 22 possible
looking for users with name Piotr in poznan
got 50 of maximum 175 possible
looking for users wi

In [106]:
save_to_file("saved_users_PL", users)

In [109]:
users = {} 

In [110]:
for country_key in list(voivodeships_data.keys()):
    users.update({country_key : {} })
    for voivodeship in voivodeships_data[country_key]:
        users[country_key].update({voivodeship["name"] : {}})
        for city in voivodeship["cities"]:
            users[country_key][voivodeship["name"]].update({city :[]})   

In [111]:
users = get_users_for_country(users, 'GB', get_popular_names("GB"))

looking for users with name David in london
got 50 of maximum 4209 possible
looking for users with name Paul in london
got 50 of maximum 3958 possible
looking for users with name John in london
got 50 of maximum 4038 possible
looking for users with name Sarah in london
got 50 of maximum 1621 possible
looking for users with name Emma in london
got 50 of maximum 1171 possible
looking for users with name Laura in london
got 50 of maximum 1126 possible
looking for users with name David in birmingham
got 50 of maximum 407 possible
looking for users with name Paul in birmingham
got 50 of maximum 404 possible
looking for users with name John in birmingham
got 50 of maximum 447 possible
looking for users with name Sarah in birmingham
got 50 of maximum 155 possible
looking for users with name Emma in birmingham
got 50 of maximum 123 possible
looking for users with name Laura in birmingham
got 50 of maximum 135 possible
looking for users with name David in glasgow
got 50 of maximum 943 possible


In [112]:
save_to_file("saved_users_GB", users)

In [ ]:
# with open("saved_users.json") as json_file:
#     users = json.load(json_file)

In [76]:
users

{'PL': {'Mazowsze': {'warsaw': [{'type': 'user',
     'id': 133933703,
     'permalink': 'https://soundcloud.com/piotr-ho',
     'name': 'Piotr Ho',
     'likeCount': 407,
     'city': 'Warsaw',
     'countryCode': 'PL',
     'gender': 'M'},
    {'type': 'user',
     'id': 20948582,
     'permalink': 'https://soundcloud.com/soundteller-records',
     'name': 'Soundteller Records Label Group',
     'likeCount': 337,
     'city': 'Warsaw',
     'countryCode': 'PL',
     'gender': 'M'},
    {'type': 'user',
     'id': 42007817,
     'permalink': 'https://soundcloud.com/piotr-dylewski',
     'name': 'Piotr "Banjhakri" Dylewski',
     'likeCount': 181,
     'city': 'Warsaw',
     'countryCode': 'PL',
     'gender': 'M'},
    {'type': 'user',
     'id': 96064522,
     'permalink': 'https://soundcloud.com/piotr-wiese',
     'name': 'Piotr Wiese',
     'likeCount': 109,
     'city': 'Warsaw',
     'countryCode': 'PL',
     'gender': 'M'},
    {'type': 'user',
     'id': 24093435,
     'permali

In [78]:
likes = {}

In [28]:
# get_user_likes('david-morris-340740716')
# user_likes = get_user_likes("ramzes-958123791")

In [79]:
likes = get_users_likes(users, likes, user_city_limit=40)

user seismics
likes of user no: 1 , 4
user anna-maria-138
likes of user no: 2 , 4
user piotr-krzysztof-st-pie
likes of user no: 3 , 4
user anna-lipiec-312392202
likes of user no: 4 , 4
user katarzyna-k-558876740
likes of user no: 5 , 4
user katarzyna-pietrasik
likes of user no: 6 , 4
user kasiut
likes of user no: 7 , 4
user maestroxmusic
likes of user no: 8 , 4
user marcin02141
likes of user no: 9 , 4
user krzysztof-kowaiczyk-1
likes of user no: 10 , 4
user marcin-makowski-1
likes of user no: 11 , 4
user kassni
likes of user no: 12 , 4
user anna-rylko
likes of user no: 13 , 4
user chybamy
likes of user no: 14 , 4
user user8977977
likes of user no: 15 , 4
user agnieszkaszczepaniak
likes of user no: 16 , 4
user that-simple
likes of user no: 17 , 4
user chlopiecie
likes of user no: 18 , 4
user anna-330007270
likes of user no: 19 , 4
user krzysztof-sobczuk-1
likes of user no: 20 , 4
user ka-jot-1
likes of user no: 21 , 4
user katarzyna-ru-36360572
likes of user no: 22 , 4
user piotr-rogals

In [69]:
save_to_file("saved_likes", likes)